In [1]:
from datetime import date
import numpy as np
import pandas as pd
from scipy.stats import norm
import cufflinks as cf

cf.go_offline()

In [119]:
N = norm.cdf
N_prime = norm.pdf

def d1(S,K,r,q,T,sigma):
    return (np.log(S/K)+(r-q+sigma**2/2)*T)/(sigma*np.sqrt(T))

def d2(S,K,r,q,T,sigma):
    return d1(S,K,r,q,T,sigma)-sigma*np.sqrt(T)

def bs_price(S,K,r,q,T,sigma,option_type):
    if option_type == 'C':
        return np.exp(-q*T)*S*N(d1(S,K,r,q,T,sigma))-np.exp(-r*T)*K*N(d2(S,K,r,q,T,sigma))
    elif option_type == 'P':
        return -np.exp(-q*T)*S*N(-d1(S,K,r,q,T,sigma))+np.exp(-r*T)*K*N(-d2(S,K,r,q,T,sigma))
    
def bs_delta(S,K,r,q,T,sigma,option_type):
    if option_type == 'C':
        return np.exp(-q*T)*N(d1(S,K,r,q,T,sigma))
    elif option_type == 'P':
        return -np.exp(-q*T)*N(-d1(S,K,r,q,T,sigma))

In [218]:
def load_warrant_historical_prices(ticker):
    df = pd.read_excel('case_study_q1_data.xlsx', sheet_name='Warrant history', skiprows=3)
    
    df.rename(columns={
        'LVWR/WS': 'LVWR/WS_date',
        'Unnamed: 1': 'LVWR/WS_price',
        'TLSIW': 'TLSIW_date',
        'Unnamed: 3': 'TLSIW_price',
        'AFRIW': 'AFRIW_date',
        'Unnamed: 5': 'AFRIW_price',
        'CLBTW': 'CLBTW_date',
        'Unnamed: 7': 'CLBTW_price',
        'GRND/WS': 'GRND/WS_date',
        'Unnamed: 9': 'GRND/WS_price',
    }, inplace=True)
    
    df = df.loc[:,df.columns.str.startswith(ticker)]
    df.dropna(inplace=True)
    
    df.set_index(ticker+'_date', inplace=True)
    df.rename_axis(index={ticker+'_date': 'date'}, inplace=True)
    
    return df

def load_stock_historical_prices(ticker):
    df = pd.read_excel('case_study_q1_data.xlsx', sheet_name='Stock history', skiprows=2)
    
    df.rename(columns={
        'LVWR': 'LVWR_date',
        'LVWR.1': 'LVWR_price',
        'TLSI': 'TLSI_date',
        'TLSI.1': 'TLSI_price',
        'AFRI': 'AFRI_date',
        'AFRI.1': 'AFRI_price',
        'CLBT': 'CLBT_date',
        'CLBT.1': 'CLBT_price',
        'GRND': 'GRND_date',
        'GRND.1': 'GRND_price',
    }, inplace=True)
    
    df = df.loc[:,df.columns.str.startswith(ticker)]
    df.dropna(inplace=True)
    
    df.set_index(ticker+'_date', inplace=True)
    df.rename_axis(index={ticker+'_date': 'date'}, inplace=True)
    
    return df

def load_warrant_static_data(ticker):
    df = pd.read_excel('case_study_q1_data.xlsx', sheet_name='Warrant static data', skiprows=1)
    
    df.rename(columns={
        'Unnamed: 1': 'field'
    }, inplace=True)
    
    df.set_index('field', inplace=True)
    
    df = df.loc[:,df.columns.str.startswith(ticker)]    
    
    return df.T

def calculate_theoretical_delta(u_ticker, w_ticker):
    
    df_und = load_stock_historical_prices(u_ticker)
    u_date = pd.to_datetime(df_und.iloc[[-1]].index.values[0]).date()
    u_price = df_und.iloc[-1,0]

    #estimate implied vol using realized volatility of stock
    df_und['return'] = np.log(df_und[u_ticker+'_price'] / df_und[u_ticker+'_price'].shift(1))
    iv = df_und['return'].dropna().std()*np.sqrt(252)

    df_wrt = load_warrant_static_data(w_ticker)
    expiry = pd.to_datetime(df_wrt['Expiry'], format='%m/%d/%Y', errors='coerce').iloc[0].date()
    strike = pd.to_numeric(df_wrt['Strike'], errors='coerce').iloc[0]
    years_to_expiry = (expiry - u_date).days / 365

    return bs_delta(S=u_price, K=strike, r=0.055, q=0, T=years_to_expiry, sigma=iv, option_type='C')

def calculate_delta_notional(eval_date, w_tkr, qty):
    df_wrt = load_warrant_static_data(w_tkr)
    expiry = pd.to_datetime(df_wrt['Expiry'], format='%m/%d/%Y', errors='coerce').iloc[0].date()
    strike = pd.to_numeric(df_wrt['Strike'], errors='coerce').iloc[0]
    years_to_expiry = (expiry - eval_date).days / 365

    d = bs_delta(S=u_price, K=strike, r=0.055, q=0, T=years_to_expiry, sigma=iv, option_type='C')    
    
    return qty * strike * d

In [137]:
#load_warrant_static_data('LVWR')
#load_stock_historical_prices('LVWR')
load_warrant_historical_prices('LVWR/WS')

,LVWR/WS_price
date,
2024-01-02,0.3999
2024-01-03,0.4000
2024-01-04,0.4490
2024-01-05,0.4489
2024-01-08,0.4130
...,...
2024-04-19,0.2599
2024-04-22,0.2525
2024-04-26,0.2244


#### 1) Quantify this security’s theoretical delta using a Black-Scholes model based on recent observable data (approximations are fine, no need to derive BS model here).

In [190]:
df_und = load_stock_historical_prices('LVWR')
u_date = pd.to_datetime(df_und.iloc[[-1]].index.values[0]).date()
u_price = df_und.iloc[-1,0]

#estimate implied vol using realized volatility of stock
df_und['return'] = np.log(df_und['LVWR_price'] / df_und['LVWR_price'].shift(1))
iv = df_und['return'].dropna().std()*np.sqrt(252)

df_wrt = load_warrant_static_data('LVWR')
expiry = pd.to_datetime(df_wrt['Expiry'], format='%m/%d/%Y', errors='coerce').iloc[0].date()
strike = pd.to_numeric(df_wrt['Strike'], errors='coerce').iloc[0]
years_to_expiry = (expiry - u_date).days / 365

print(f"Black-Scholes delta: {bs_delta(S=u_price, K=strike, r=0.055, q=0, T=years_to_expiry, sigma=iv, option_type='C'):0.2f}")

Black-Scholes delta: 0.62


#### 2) Quantify this security’s observed (realized) delta over recent history

In [194]:
df_und = load_stock_historical_prices('LVWR')
#df_und['LVWR_return'] = np.log(df_und['LVWR_price'] / df_und['LVWR_price'].shift(1))
df_und['LVWR_change'] = df_und['LVWR_price'] - df_und['LVWR_price'].shift(1)
df_wrt = load_warrant_historical_prices('LVWR/WS')
#df_wrt['LVWR/WS_return'] = np.log(df_wrt['LVWR/WS_price'] / df_wrt['LVWR/WS_price'].shift(1))
df_wrt['LVWR/WS_change'] = df_wrt['LVWR/WS_price'] - df_wrt['LVWR/WS_price'].shift(1)

df = df_und.join(df_wrt, how='left')
df.ffill(inplace=True)
df.dropna(inplace=True)

#df[['LVWR/WS_change', 'LVWR_change']].iplot(kind='bar')

df['rlzd_delta'] = df.loc[df['LVWR_change'] != 0,'LVWR/WS_change'].div(df.loc[df['LVWR_change'] != 0, 'LVWR_change'])

df.fillna(0., inplace=True)

#df['rlzd_delta'].iplot(kind='bar', title='ratio of price changes')

print(f"realized delta: {df['rlzd_delta'].mean():0.2f}")


realized delta: 0.06


#### 3) How do these measures compare?

In [196]:
print(f"realized delta is 10% of theoretical delta")

realized delta is 10% of theoretical delta


#### 4) What factors do you believe are affecting security pricing?

In [201]:
n_obs_wrt = load_warrant_historical_prices('LVWR/WS').shape[0]
n_obs_und = load_stock_historical_prices('LVWR').shape[0]

print(f"Warrant volume is likely very low relative to underlying shares.  There were {n_obs_und - n_obs_wrt} days when the underlying shares traded and the warrants did not")

Warrant volume is likely very low relative to underlying shares.  There were 14 days when the underlying shares traded and the warrants did not


#### 5) Quantify the theoretical delta 1% risk of this portfolio using real world inputs. Please detail any assumptions that you make

In [231]:
pd.options.display.float_format = "{:0,.2f}".format

positions = {
    'LVWR/WS': ['LVWR', 'LVWR/WS', 1e6],
    'TLSIW': ['TLSI', 'TLSIW', 1e6],
    'AFRIW': ['AFRI', 'AFRIW', 1e6],
    'CLBTW': ['CLBT', 'CLBTW', 1e6],
    'GRND/WS': ['GRND', 'GRND/WS', 1e6],
}

results = {}

for name, pos in positions.items():
    
    
    u_tkr = pos[0]
    w_tkr = pos[1]
    w_qty = pos[2]
    
    df_wrt = load_warrant_static_data(w_tkr)
    expiry = pd.to_datetime(df_wrt['Expiry'], format='%m/%d/%Y', errors='coerce').iloc[0].date()
    strike = pd.to_numeric(df_wrt['Strike'], errors='coerce').iloc[0]
    years_to_expiry = (expiry - u_date).days / 365
    
    results[name] = [
        
        expiry,
        strike,
        calculate_theoretical_delta(u_tkr, w_tkr),
        calculate_delta_notional(date(2024,4,30), w_tkr, w_qty)
    ]
    
    

port = pd.DataFrame.from_dict(results, orient='index', columns=['expiry', 'strike', 'delta', 'exposure'])

display(port)

print(f"Portfolio delta 1%: {port[['exposure']].sum().iloc[0] * 0.01:0,.0f}")

,expiry,strike,delta,exposure
LVWR/WS,2027-09-27,11.50,0.62,"7,135,867.43"
TLSIW,2028-08-10,11.50,0.70,"7,732,268.69"
AFRIW,2027-06-09,11.50,0.64,"6,890,762.29"
CLBTW,2026-08-30,11.50,0.66,"6,125,436.17"
GRND/WS,2027-11-18,11.50,0.66,"7,243,886.49"


Portfolio delta 1%: 351,282


#### 6) Would the answer change if you were asked for the delta 10% or delta 20%? If so, why?

In [232]:
print(f"The exposure of the warrants portfolio is positively convex with respect to an increase in the underlying shares.  As the underlying move is increased the difference between estimated and true delta will increase.")

The exposure of the warrants portfolio is positively convex with respect to an increase in the underlying shares.  As the underlying move is increased the difference between estimated and true delta will increase.


#### 7) What has the observable (realized) $ delta of this portfolio been YTD?

In [233]:
pd.options.display.float_format = "{:0,.2f}".format

positions = {
    'LVWR/WS': ['LVWR', 'LVWR/WS', 1e6],
    'TLSIW': ['TLSI', 'TLSIW', 1e6],
    'AFRIW': ['AFRI', 'AFRIW', 1e6],
    'CLBTW': ['CLBT', 'CLBTW', 1e6],
    'GRND/WS': ['GRND', 'GRND/WS', 1e6],
}

results = {}

for name, pos in positions.items():
    
    
    u_tkr = pos[0]
    w_tkr = pos[1]
    w_qty = pos[2]
    
    df_wrt = load_warrant_static_data(w_tkr)
    expiry = pd.to_datetime(df_wrt['Expiry'], format='%m/%d/%Y', errors='coerce').iloc[0].date()
    strike = pd.to_numeric(df_wrt['Strike'], errors='coerce').iloc[0]
    years_to_expiry = (expiry - u_date).days / 365
    
    results[name] = [
        
        expiry,
        strike,
        calculate_theoretical_delta(u_tkr, w_tkr),
        calculate_delta_notional(date(2024,4,30), w_tkr, w_qty)
    ]

df_und = load_stock_historical_prices('LVWR')
df_und['LVWR_change'] = df_und['LVWR_price'] - df_und['LVWR_price'].shift(1)
df_wrt = load_warrant_historical_prices('LVWR/WS')
df_wrt['LVWR/WS_change'] = df_wrt['LVWR/WS_price'] - df_wrt['LVWR/WS_price'].shift(1)

df = df_und.join(df_wrt, how='left')
df.ffill(inplace=True)
df.dropna(inplace=True)

#df[['LVWR/WS_change', 'LVWR_change']].iplot(kind='bar')

df['rlzd_delta'] = df.loc[df['LVWR_change'] != 0,'LVWR/WS_change'].div(df.loc[df['LVWR_change'] != 0, 'LVWR_change'])

df.fillna(0., inplace=True)

#df['rlzd_delta'].iplot(kind='bar', title='ratio of price changes')

print(f"realized delta: {df['rlzd_delta'].mean():0.2f}")

realized delta: 0.06
